4) отсеять исполнителей, у которых меньше чем k треков (прикинуть количество из распределения слов на статье на пудинге)
5) проверить доступность лирикса на Genius для каждого исполнителя, динамически отсеивая тех, для которых не набралось k песен с лириксом

In [2]:
import pickle
import re
import time
from os import listdir

import numpy as np
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt

In [67]:
artists_info = pd.read_csv(
    "../data/generated/1.2-spotify_artists_info/artists_info.csv"
)

In [68]:
import pickle

retrieved = []
for dir_i in range(1, 6):
    path_to_dir = f"../scripts/1.2-spotify_artists_albums_extraction/extracted_albums_info/pt_{dir_i}"
    files = listdir(path_to_dir)
    for pickle_name in files:
        with open(f"{path_to_dir}/{pickle_name}", "rb") as f:
            pickle_part_ = pickle.load(f)
            retrieved.append(pickle_part_)

path_to_missings = (
    "../scripts/1.2-spotify_artists_albums_extraction/extracted_albums_info/missings"
)
for missing_pickle in listdir(path_to_missings):
    with open(f"{path_to_missings}/{missing_pickle}", "rb") as f:
        pickle_part_ = pickle.load(f)
        retrieved.append(pickle_part_)

retrieved_df = pd.concat(retrieved)

In [71]:
tracks_n_nums = (
    retrieved_df[["artist_id", "n_tracks"]].groupby("artist_id").count().n_tracks.values
)
cut_off_words = 20000
cut_off_tracks = np.floor(cut_off_words / 700)

print(
    f"By setting a cut-off point as {cut_off_words} words we lose {np.round((tracks_n_nums < cut_off_tracks).mean()*100, 2)}% of the artists"
)

By setting a cut-off point as 20000 words we lose 79.11% of the artists


In [6]:
artist_albums_agg = retrieved_df[["artist_id", "n_tracks"]].groupby("artist_id").count()
legit_ids = artist_albums_agg[artist_albums_agg["n_tracks"] >= cut_off_tracks].index
artists_albums_min_words = retrieved_df[retrieved_df["artist_id"].isin(legit_ids)]

In [7]:
artists_albums_min_words["artist_id"].unique()
artists_info[artists_info["artist_name"] == "Preto Show"]

,genre,artist_name,artist_id,followers,genres,image_url,popularity,is_valid_genre
4513,hip hop tuga,Preto Show,4YlvKlWWHJl1TuXvrk94OW,43266,hip hop tuga||kizomba||musica angolana||rap an...,https://i.scdn.co/image/ab6761610000e5ebffc89b...,39,True


In [8]:
total_tracks = (
    retrieved_df[["artist_id", "n_tracks"]].groupby("artist_id").sum().reset_index()
)
artists_to_retrieve_ids = total_tracks[total_tracks["n_tracks"] > 20]["artist_id"]
artists_to_retrieve_names = artists_info[
    artists_info["artist_id"].isin(artists_to_retrieve_ids)
]

In [180]:
gen_api_client = lyricsgenius.Genius(
    "69ryL9iFYcVyyotx4dwu0jHM7w6epbNcUije9veLy-eM1V0hH1KLU1dwSDsTlrgp", verbose=False
)

In [12]:
album_ = gen_api_client.search_album("Кривой эфир", "ATL")

In [73]:
merged_full_artists = pd.merge(
    retrieved_df, artists_info[["artist_name", "artist_id"]], on="artist_id"
)
merged_full_artists = merged_full_artists[merged_full_artists["release_date"] != "0000"]
artists_ids_list_full = merged_full_artists["artist_id"].unique()

In [75]:
merged_full_artists[merged_full_artists["artist_id"] == "31L9u9oYb9kZIrkrCAxThc"]

,artist_id,album_id,album_name,album_type,artists,release_date,n_tracks,artist_name
128519,31L9u9oYb9kZIrkrCAxThc,1qFHduYUn2R2Gg8wpjc2Is,Kemotherapy I,album,31L9u9oYb9kZIrkrCAxThc,2021-09-09,12,Bull Dogg
128520,31L9u9oYb9kZIrkrCAxThc,7tgDEr4237vUKCp37qORap,Mood,single,31L9u9oYb9kZIrkrCAxThc,2021-07-07,1,Bull Dogg
128521,31L9u9oYb9kZIrkrCAxThc,3Oqja3LreapjbkwpmhE6RN,Byose Kuri Jah,single,31L9u9oYb9kZIrkrCAxThc,2020-07-14,3,Bull Dogg


In [14]:
album.__dict__["tracks"][0].__dict__["song"].__dict__["lyrics_state"]

NameError: name 'album' is not defined

In [ ]:
album = gen_api_client.search_album(name="Amarelo", artist="Emicida")

In [ ]:
track_mixed = gen_api_client.search_song(title="Konstrukt", artist="Oxxxymiron, Porchy")

In [ ]:
track_mixed.__dict__["_body"]["language"]

'ru'

In [ ]:
album_bad = gen_api_client.search_album(
    name="Be More Chill", artist="Be More Chill Ensemble"
)

In [ ]:
album_bad.__dict__["tracks"][0].__dict__["_body"]["song"]["instrumental"]

True

In [ ]:
album_.__dict__["tracks"][0].__dict__["song"].__dict__["_body"].keys()

dict_keys(['_type', 'annotation_count', 'api_path', 'artist_names', 'full_title', 'header_image_thumbnail_url', 'header_image_url', 'id', 'instrumental', 'language', 'lyrics_owner_id', 'lyrics_state', 'lyrics_updated_at', 'path', 'pyongs_count', 'relationships_index_url', 'release_date_components', 'release_date_for_display', 'release_date_with_abbreviated_month_for_display', 'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title', 'title_with_featured', 'updated_by_human_at', 'url', 'featured_artists', 'primary_artist'])

In [ ]:
album_.__dict__["tracks"][0].__dict__["song"].__dict__["_body"]["lyrics"]

KeyError: 'lyrics'

In [ ]:
album_.__dict__["tracks"][0].__dict__["song"].__dict__["_body"]["title"]

'Кривой эфир (Curve Broadcast)'

In [ ]:
album_.__dict__["tracks"][0].__dict__["song"].__dict__["_body"]["title_with_featured"]

'Кривой эфир (Curve Broadcast)'

In [ ]:
album_.__dict__["tracks"][0].__dict__["_body"]["song"].keys()

dict_keys(['_type', 'annotation_count', 'api_path', 'artist_names', 'full_title', 'header_image_thumbnail_url', 'header_image_url', 'id', 'instrumental', 'language', 'lyrics_owner_id', 'lyrics_state', 'lyrics_updated_at', 'path', 'pyongs_count', 'relationships_index_url', 'release_date_components', 'release_date_for_display', 'release_date_with_abbreviated_month_for_display', 'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title', 'title_with_featured', 'updated_by_human_at', 'url', 'featured_artists', 'primary_artist'])

In [ ]:
track_mixed.__dict__["_body"]["primary_artist"]

{'_type': 'artist',
 'api_path': '/artists/1167808',
 'header_image_url': 'https://images.genius.com/5f5903b50ee061570c48a30f727af0ab.1000x432x1.jpg',
 'id': 1167808,
 'image_url': 'https://images.genius.com/3e7ad2e3223bca6ab39fc3a34f5a3408.1000x1000x1.jpg',
 'index_character': 'b',
 'is_meme_verified': False,
 'is_verified': False,
 'name': 'Booking Machine',
 'slug': 'Booking-machine',
 'url': 'https://genius.com/artists/Booking-machine'}

In [ ]:
album_.__dict__["_body"]["name"]

'Кривой эфир (Distorted Broadcast)'

In [ ]:
def is_track_ok(track):
    song_dict = track.__dict__["song"].__dict__
    if song_dict["lyrics_state"] != "complete":
        return False
    if song_dict["_body"]["instrumental"]:
        return False
    if song_dict["_body"]["language"] != "en":
        return False
    if not song_dict["lyrics"]:
        return False

    return True


def process_track_data(track, is_single=False):
    if is_single:
        song_dict = track.__dict__
    else:
        song_dict = track.__dict__["song"].__dict__

    # track data
    lyrics = song_dict["lyrics"]
    annotation_count = song_dict["_body"]["annotation_count"]
    title_ = song_dict["_body"]["title"]
    title_with_featured = song_dict["_body"]["title_with_featured"]
    track_id = song_dict["_body"]["id"]

    _stats = song_dict["_body"]["stats"]
    try:
        is_hot = _stats["hot"]
    except KeyError:
        is_hot = None

    try:
        pageviews = _stats["pageviews"]
    except KeyError:
        pageviews = None

    # artist data

    primary_artist_raw = song_dict["_body"]["primary_artist"]
    primary_artist_id = primary_artist_raw["id"]
    primary_artist_name = primary_artist_raw["name"]

    featured_artists_raw = song_dict["_body"]["featured_artists"]
    if featured_artists_raw:
        featured_artists = [
            [i["_type"], i["id"], i["name"]] for i in featured_artists_raw
        ]
    else:
        featured_artists = []

    return [
        [
            track_id,
            title_,
            title_with_featured,
            annotation_count,
            is_hot,
            pageviews,
            lyrics,
            primary_artist_id,
            primary_artist_name,
        ],
        featured_artists,
    ]


def collect_album_data(artist_name, album_name, gen_api):
    album_data = gen_api.search_album(name=album_name, artist=artist_name)
    return album_data


def process_album_data(album_, *args):
    album_tracks = album_.__dict__["tracks"]
    n_tracks_album = len(album_.__dict__["tracks"])
    album_data = []
    for track in album_tracks:
        if not is_track_ok(track):
            continue

        track_info_genius, featured_artists = process_track_data(track)
        album_data.append(
            [track_info_genius, featured_artists, args]
        )  # artist_id_spotify, album_name_spotify, album_type_spotify,
    return album_data, n_tracks_album


def collect_single_data(artist_name, single_name, gen_api):
    single_data = gen_api.search_song(title=single_name, artist=artist_name)
    return single_data

In [ ]:
merged_full_artists.sort_values(by="release_date", ascending=False)

,artist_id,album_id,album_name,album_type,artists,release_date,n_tracks,artist_name
405918,17kX2iCte2ZwbtwhIF7bBq,7HUNKLZXUpf9PNmihCjg6a,Porta (Freestyle),single,17kX2iCte2ZwbtwhIF7bBq,2023-06-06,1,Black K
163410,0kEfub5RzlZOB2zGomqVSU,4ShnBsLIY7i42sluMXvn9U,The Comment (Mixes),single,0kEfub5RzlZOB2zGomqVSU|7lgauJ5ZMIIwrJwvUvrWC4,2023-05-26,2,Gil Scott-Heron
163411,0kEfub5RzlZOB2zGomqVSU,2TrdvdDY8MzA8YcePXsNhZ,Whitey On The Moon,single,0kEfub5RzlZOB2zGomqVSU|7lgauJ5ZMIIwrJwvUvrWC4,2023-05-26,2,Gil Scott-Heron
1408,0FUsrstJwmg4WVHQMTYuUA,71kPiLmNbaT0eylc5Ycvdj,Prenses (Remix),single,0FUsrstJwmg4WVHQMTYuUA,2023-05-03,1,Şehinşah
150779,6XQrv3AiNUS61JFK1VITTU,3yvd16lFMNCqDoiaqjo0Qx,Fragmentado,single,62sKdgq98PB8klSTM8Qvaw|6XQrv3AiNUS61JFK1VITTU|...,2023-04-07,1,Jean Tassy
...,...,...,...,...,...,...,...,...
207535,3ctWmmsiRK0L4BsQsfYJ4V,0ivNYmsFWwW7COWIin7S1k,Obiecana Ziemia,album,3ctWmmsiRK0L4BsQsfYJ4V,1900,15,Thinkadelic
207534,3ctWmmsiRK0L4BsQsfYJ4V,7gW25CIhTdWOZJUsIbSlyV,Obiecana Ziemia,album,3ctWmmsiRK0L4BsQsfYJ4V,1900,15,Thinkadelic
207378,69hBq9L22B5LOu2gkbBRlb,4Zs14OydKIxTwcU6Y7ZI0Z,Ironia,album,69hBq9L22B5LOu2gkbBRlb,1900,15,Pareslow
207379,69hBq9L22B5LOu2gkbBRlb,4N7t5W4ZfkLlJ6WKT3KaOI,Galaktao,album,69hBq9L22B5LOu2gkbBRlb,1900,20,Pareslow


In [ ]:
artists_ids_list_full

array(['0DOixxvuN7cxZ7YiyMzlOt', '3DOJU0CqEqSdfFIMGiykRv',
       '71SuRO9s3mJNnwAlw7TD7l', ..., '1qOsx30xuEUpA6XpJSsoIN',
       '4vb1fNDFbTmCBI3VINepk7', '6nELoJ6eMXfYHX5xocKf33'], dtype=object)

In [ ]:
album_bad

Album(id, artist, ...)

In [27]:
merged_full_artists

,artist_id,album_id,album_name,album_type,artists,release_date,n_tracks,artist_name
0,0DOixxvuN7cxZ7YiyMzlOt,0kQduCUJJoJsKVJ6zr67nB,Lisboa Azul (Original Series Soundtrack),album,0DOixxvuN7cxZ7YiyMzlOt,2019-10-23,19,Johnny Virtus
1,0DOixxvuN7cxZ7YiyMzlOt,6FjefVWpWhfmrOIX7SKKvl,UniVersos,album,0DOixxvuN7cxZ7YiyMzlOt,2012-01-21,20,Johnny Virtus
2,0DOixxvuN7cxZ7YiyMzlOt,15G7TOQBrQD0huFgZMIXKQ,Não Posso Agora,single,0DOixxvuN7cxZ7YiyMzlOt,2023-02-10,1,Johnny Virtus
3,0DOixxvuN7cxZ7YiyMzlOt,148fCYh6wwJ3EQ1L5nDSG3,Sicklos,single,0DOixxvuN7cxZ7YiyMzlOt,2022-06-29,1,Johnny Virtus
4,0DOixxvuN7cxZ7YiyMzlOt,6vP5NkRvMajnbHh3crLRns,Conversa em Dia,single,0DOixxvuN7cxZ7YiyMzlOt,2021-02-12,1,Johnny Virtus
...,...,...,...,...,...,...,...,...
612477,6nELoJ6eMXfYHX5xocKf33,2FzUHjvyicfBWDD5o534DH,Prosper,single,6nELoJ6eMXfYHX5xocKf33,2021-04-09,1,S.O.
612478,6nELoJ6eMXfYHX5xocKf33,4cam4h7xk2aA2QJAYE0PhD,Closer,single,455WReaTbCB1fEjWrV29x4|6nELoJ6eMXfYHX5xocKf33,2021-03-12,1,S.O.
612479,6nELoJ6eMXfYHX5xocKf33,4EVqXVMvFc6JDLNWyxUeww,All I Know,single,4w00ajMRfMhWzm7a88Hlon|6nELoJ6eMXfYHX5xocKf33|...,2021-02-26,1,S.O.
612480,6nELoJ6eMXfYHX5xocKf33,05hG4ktRZrQMWiZvVssJsC,Pray for Me,single,6RPalPy5qqMN2iHkKEVYFu|0gOMnWbGP52F3paKcp5Ts2|...,2020-06-12,1,S.O.


True

In [16]:
ids_split = np.array_split(artists_ids_list_full, 5)
for i, split_ in enumerate(ids_split):
    df_split = merged_full_artists[merged_full_artists["artist_id"].isin(split_)]
    with open(
        f"../data/generated/1.3-genius_ids_to_load/artists_ids_pt{i+1}.pickle", "wb"
    ) as f, open(
        f"../data/generated/1.3-genius_ids_to_load/artists_and_albums_ids_pt{i+1}.pickle",
        "wb",
    ) as f1:
        pickle.dump(split_, f)
        pickle.dump(df_split, f1)

In [ ]:
# TODO: 1) be sure that I also save the data related to initial query (album name, artist name)
# 2) make sure I also include the artists which are in .feat (artists in retrieved_df) - hopefully, \
# I won't need to post-load additional data related to authentic names from Spotify
# 3) create 5 separate scripts with separate tokens for lyrics retrieval

for artist_id in artists_ids_list_full:
    artist_discography = {artist_id: []}

    sub_df = merged_full_artists[merged_full_artists["artist_id"] == artist_id]
    sub_df = sub_df.sort_values(by="release_date", ascending=False)
    artist_name = sub_df["artist_name"]
    data_saved = []

    tracks_collected = 0
    tracks_traversed = 0

    tracks_max = sub_df["n_tracks"].sum()
    for i, df_row in sub_df.iterrows():
        album_name = df_row["album_name"]
        album_type = df_row["album_type"]
        tracks_traversed += df_row["n_tracks"]

        if album_type == "album":
            album_data = collect_album_data(artist_name, album_name, gen_api)
            if not album_data:
                continue
            album_info, n_tracks = process_album_data(
                album_data, album_name, album_type
            )

        elif album_type == "single":
            single_data = collect_single_data(artist_name, album_name, gen_api)
            if not single_data:
                continue
            album_info = [
                process_track_data(single_data, is_single=True)
                + [album_name, album_type]
            ]
            n_tracks = 1

        tracks_collected += n_tracks

        artist_discography[artist_id].append([album_info, n_tracks])
        if tracks_max - tracks_traversed + tracks_collected < 15:
            # add to the list of unfound artists and report how many albums were actually found
            # TODO: Maybe save the data anyway to a separate directory
            break

        if tracks_collected > 65:
            with open(f"{artist_id}.pickle", "wb") as f:
                pickle.dump(artist_discography, f)
            break

for legit_artist in list_artsists:
    data_saved = []

    tracks_collected = 0
    tracks_traversed = 0
    tracks_max = max_tracks(legit_artist)
    for album_name in album_list(legit_artist):
        
        album_data = collect_album_data(legit_artist, album_name)
        n_tracks_album = get_n_tracks(album_name)
        for track in album_tracks:
            if not_ok(track):
                continue

            # lyrics_state
            lyrics = collect_lyrics(track)
            other_info = collect_info(track)
            data_save.append(lyrics, **other_info)
            tracks_collected += 1
        
        tracks_traversed += n_tracks_album

        if tracks_max - tracks_traversed + tracks_collected < 15:
            break
        
        if tracks_collected > 65:
            save_data
            break
            
    

In [34]:
import json

PT = "1"
LEFT_SUFFIX = ""
EXTRACTION_PATH = "../data/generated/1.3-genius_ids_to_load"

with open("../tokens/genius_tokens.json", "r") as f:
    genius_tokens = json.load(f)
    genius_access_token = genius_tokens[PT]["access_token"]


with open(
    f"{EXTRACTION_PATH}/artists_ids_pt{PT}{LEFT_SUFFIX}.pickle", "rb"
) as f1, open(
    f"{EXTRACTION_PATH}/artists_and_albums_ids_pt{PT}{LEFT_SUFFIX}.pickle", "rb"
) as f2:
    artists_ids = pickle.load(f1)
    df_artists_albums = pickle.load(f2)
gen_api = lyricsgenius.Genius(genius_access_token, verbose=False)

In [26]:
def is_track_ok(track):
    song_dict = track.__dict__["song"].__dict__
    if song_dict["lyrics_state"] != "complete":
        return False
    if song_dict["_body"]["instrumental"]:
        return False
    if song_dict["_body"]["language"] != "en":
        return False
    if not song_dict["lyrics"]:
        return False

    return True


def process_track_data(track, is_single=False):
    if is_single:
        song_dict = track.__dict__
    else:
        song_dict = track.__dict__["song"].__dict__

    # track data
    lyrics = song_dict["lyrics"]
    annotation_count = song_dict["_body"]["annotation_count"]
    title_ = song_dict["_body"]["title"]
    title_with_featured = song_dict["_body"]["title_with_featured"]
    track_id = song_dict["_body"]["id"]

    _stats = song_dict["_body"]["stats"]
    try:
        is_hot = _stats["hot"]
    except KeyError:
        is_hot = None

    try:
        pageviews = _stats["pageviews"]
    except KeyError:
        pageviews = None

    # artist data

    primary_artist_raw = song_dict["_body"]["primary_artist"]
    primary_artist_id = primary_artist_raw["id"]
    primary_artist_name = primary_artist_raw["name"]

    featured_artists_raw = song_dict["_body"]["featured_artists"]
    if featured_artists_raw:
        featured_artists = [
            [i["_type"], i["id"], i["name"]] for i in featured_artists_raw
        ]
    else:
        featured_artists = []

    return [
        [
            track_id,
            title_,
            title_with_featured,
            annotation_count,
            is_hot,
            pageviews,
            lyrics,
            primary_artist_id,
            primary_artist_name,
        ],
        featured_artists,
    ]


def collect_album_data(artist_name, album_name, gen_api):
    album_data = gen_api.search_album(name=album_name, artist=artist_name)
    return album_data


def process_album_data(album_, *args):
    album_tracks = album_.__dict__["tracks"]
    n_tracks_album = len(album_.__dict__["tracks"])
    album_data = []
    for track in album_tracks:
        if not is_track_ok(track):
            continue

        track_info_genius, featured_artists = process_track_data(track)
        album_data.append(
            [track_info_genius, featured_artists, args]
        )  # artist_id_spotify, album_name_spotify, album_type_spotify,
    return album_data, n_tracks_album


def collect_single_data(artist_name, single_name, gen_api):
    single_data = gen_api.search_song(title=single_name, artist=artist_name)
    return single_data

In [42]:
artists_ids_left = artists_ids.copy()
df_artists_albums_left = df_artists_albums.copy()

for artist_id in artists_ids:
    artist_discography = {artist_id: []}
    sub_df = df_artists_albums[df_artists_albums["artist_id"] == artist_id]
    sub_df = sub_df.sort_values(by="release_date", ascending=False)
    artist_name = sub_df["artist_name"].iloc[0]
    data_saved = []
    saved_full = False

    tracks_collected = 0
    tracks_traversed = 0

    tracks_max = sub_df["n_tracks"].sum()
    for i, df_row in sub_df.iterrows():
        album_name = df_row["album_name"]
        if album_name != "UniVersos":
            continue

        album_type = df_row["album_type"]
        tracks_traversed += df_row["n_tracks"]

        if album_type == "album":
            album_data = collect_album_data(artist_name, album_name, gen_api)
            if not album_data:
                continue
            album_info, n_tracks = process_album_data(
                album_data, album_name, album_type
            )

        elif album_type == "single":
            single_data = collect_single_data(artist_name, album_name, gen_api)
            if not single_data:
                continue
            album_info = [
                process_track_data(single_data, is_single=True)
                + [album_name, album_type]
            ]
            n_tracks = 1

        tracks_collected += n_tracks

        artist_discography[artist_id].append([album_info, n_tracks])
        if tracks_max - tracks_traversed + tracks_collected < 15:
            with open(
                f"genius_track_info_extracted/artists_not_enough_data/{artist_id}.pickle",
                "wb",
            ) as f:
                pickle.dump(artist_discography, f)
            break

        if tracks_collected > 65:
            with open(
                f"genius_track_info_extracted/genius_artists_pt{PT}/{artist_id}.pickle",
                "wb",
            ) as f:
                pickle.dump(artist_discography, f)
            saved_full = True
            break
    if not saved_full:
        with open(
            f"genius_track_info_extracted/artists_not_enough_data/{artist_id}.pickle",
            "wb",
        ) as f:
            pickle.dump(artist_discography, f)
    artists_ids_left = np.delete(artists_ids, 0)
    df_artists_albums_left = df_artists_albums[
        df_artists_albums["artist_id"] != artist_id
    ]

    break

In [84]:
artists_pt1 = pd.read_pickle(
    "../scripts/1.3-genius_info_extraction/genius_track_info_extracted/genius_artists_pt1/2d9LRvQJnAXRijqIJDDs2K.pickle"
)

In [161]:
to_df = []
columns = [""]
# for album_ in artists_pt1['2d9LRvQJnAXRijqIJDDs2K']:
#     album_info, n_tracks = album_
# track_info_genius, featured_artists, args]) # artist_id_spotify, album_name_spotify, album_type_spotify,
album_data = artists_pt1["2d9LRvQJnAXRijqIJDDs2K"][0][0][0]
# track_id, title_, title_with_featured, annotation_count, is_hot, pageviews, lyrics, primary_artist_id, primary_artist_name
artists_pt1["2d9LRvQJnAXRijqIJDDs2K"][0]

[[[[255844,
    'País do Futebol',
    'País do Futebol (Ft.\xa0Emicida)',
    28,
    False,
    8441,
    'País do Futebol LyricsLetra de "País do Futebol" com MC Guimê e Emicida\n\n[Introdução do Clipe Oficial]\nEmicida: A pior barreira memo é auto estima, tá ligado?\nÉ porque, mano, sem maldade, quando você nasce num lugar como esse aqui, mano, as pessoas te empurra pra baixo memo. A sociedade inteira te empurra pra baixo, pra você não acreditar memo. Nóis tem que ir pro mundo e mostrar nossa verdade, contar nossa história, tá ligado?\nIsso foi uma coisa que eu sempre acreditei desde o começo. Eu não sabia as ferramenta necessária e a música foi a ferramenta que eu encontrei pra, mano, mostrar pro mundo a minha verdade. E é muito parecido com o esporte, tá ligado? A grande maioria dos atleta aí, os melhor, os mais foda vem daqui. Os músico mais foda também vem daqui\n\nGuimê: É 0,1 % que hoje acredita, por quê? Porque a sociedade oferece isso aí pra molecada. E da onde sai os melho

In [2]:
import pandas as pd

In [2]:
sz = pd.read_pickle(
    "../scripts/1.3-genius_info_extraction/genius_track_info_extracted/genius_artists_pt2/2SrSdSvpminqmStGELCSNd.pickle"
)

In [3]:
# a = pd.read_pickle("../scripts/1.3-genius_info_extraction/genius_track_info_extracted/artists_not_enough_data/0wgUke31RMnwy7q3CdhBIi.pickle")
ids = pd.read_pickle(
    "../data/generated/1.3-genius_ids_to_load/artists_ids_pt4_left.pickle"
)
b = pd.read_pickle(
    "../data/generated/1.3-genius_ids_to_load/artists_and_albums_ids_pt4.pickle"
)

In [14]:
b[b["album_id"] == "7uYMZWwSFI1CvaK4zceOJK"]

,artist_id,album_id,album_name,album_type,artists,release_date,n_tracks,artist_name
442986,2wiZdRBsUPUbrmt5v24Y71,7uYMZWwSFI1CvaK4zceOJK,Isolation,album,2wiZdRBsUPUbrmt5v24Y71,2020-06-28,14,Fox


In [1]:
import lyricsgenius

gen_api_client = lyricsgenius.Genius(
    "69ryL9iFYcVyyotx4dwu0jHM7w6epbNcUije9veLy-eM1V0hH1KLU1dwSDsTlrgp", verbose=True
)

In [2]:
a = gen_api_client.search_album("Isolation", "Fox")

Searching for "Isolation" by Fox...


In [4]:
b[b["album_id"] == "0OAgRLB9nS0n9YImE9sspX"]

,artist_id,album_id,album_name,album_type,artists,release_date,n_tracks,artist_name
1711,7Ml6UQvSODdViugwulEjWk,0OAgRLB9nS0n9YImE9sspX,Anlamayanlar da'Var (64Bars),single,0JnDkp29FlLpGwsnub400t|7Ml6UQvSODdViugwulEjWk,2022-04-08,1,Da Poet
2186,0JnDkp29FlLpGwsnub400t,0OAgRLB9nS0n9YImE9sspX,Anlamayanlar da'Var (64Bars),single,0JnDkp29FlLpGwsnub400t|7Ml6UQvSODdViugwulEjWk,2022-04-08,1,Kamufle


In [6]:
from queue import Queue

In [7]:
a = Queue

TypeError: put() missing 1 required positional argument: 'item'